In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
from matplotlib.patches import FancyArrowPatch
import streamlit as st

# Streamlit app title
st.title("Improved Co-Funding Sponsors Network with Curved Edges")

# File upload option for user to provide CSV
uploaded_file = st.file_uploader("Upload your CSV file", type="csv")
if uploaded_file:
    # Load the dataset
    df = pd.read_csv(uploaded_file)

    # Initialize a graph
    G = nx.Graph()

    # Add edges from the dataset
    for index, row in df.iterrows():
        G.add_edge(row['Sponsor 1'], row['Sponsor 2'])

    # Use a layout for better visualization with more spacing
    pos = nx.spring_layout(G, seed=42, k=0.3)  # `k` controls node spacing

    # Calculate node degrees and scale sizes
    degree_dict = dict(G.degree())
    node_sizes = [v * 100 for v in degree_dict.values()]  # Scale node sizes by degree

    # Color nodes by degree
    norm = mcolors.Normalize(vmin=min(degree_dict.values()), vmax=max(degree_dict.values()))
    cmap = plt.cm.plasma  # Choose a colormap
    node_colors = [cmap(norm(degree_dict[node])) for node in G.nodes]

    # Create the plot
    fig, ax = plt.subplots(figsize=(20, 15))

    # Draw nodes
    node_collection = nx.draw_networkx_nodes(G, pos, ax=ax, node_size=node_sizes, node_color=node_colors, cmap=cmap)

    # Draw curved edges
    for u, v in G.edges():
        path = FancyArrowPatch(
            posA=pos[u],
            posB=pos[v],
            connectionstyle="arc3,rad=0.2",  # Adjust curvature with 'rad'
            color='gray',
            alpha=0.5,
            linewidth=0.8
        )
        ax.add_patch(path)

    # Draw labels with improved readability
    labels = {node: node for node in G.nodes}
    nx.draw_networkx_labels(
        G,
        pos,
        labels=labels,
        font_size=8,
        font_color='black',
        font_weight='bold',
        verticalalignment='center',
        horizontalalignment='center',
        ax=ax
    )

    # Add a colorbar linked to the node colors
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, shrink=0.8, ax=ax)
    cbar.set_label("Node Degree", fontsize=12)

    # Add title
    ax.set_title("Improved Co-Funding Sponsors Network with Curved Edges", fontsize=18)
    ax.axis("off")

    # Display the plot in Streamlit
    st.pyplot(fig)

    # Find the node with the highest degree
    highest_node = max(degree_dict, key=degree_dict.get)
    highest_value = degree_dict[highest_node]

    # Display results
    st.write(f"**The node with the highest degree:** `{highest_node}`")
    st.write(f"**The degree of the node:** `{highest_value}`")
    streamlit run network_visualization.py